In [ ]:
#====================#
#  Import Packages   #
#====================#

import math 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
from scipy.stats import beta

import uproot3 as uproot

# BDT
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc, recall_score, precision_score, average_precision_score, precision_recall_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_sample_weight
import joblib
import xgboost as xgb
import shap
from matplotlib.pylab import rcParams
import plotly.express as px
import seaborn as sns
from iminuit import Minuit
import ROOT
from tqdm import tqdm

## First to read in the flat caf files and the trees, mainly the genie truth and the rec trees.

In [ ]:
#========================================================#
#  A function for creating dataframes and importing data #
#  from root files                                       #
#========================================================#
def new_df(file):
    
    
    #———————————————————————————————————————————————————————————————————#
    # First, define variables needed from the genie truth and rec trees #
    #———————————————————————————————————————————————————————————————————#

    genieEvtRec_vars = ["GenieEvtRec.StdHepPdg", 
                        "GenieEvtRec.StdHepX4", 
                        "GenieEvtRec.StdHepP4",
                        "GenieEvtRec.EvtVtx"]

    rec_vars = ["rec.slc.truth.iscc",
                "rec.mc.nu.iscc"]
    
    #————————————————————————————————————
    # import the trees from the root file
    #————————————————————————————————————
    
    T_rec = uproot.open(file)['recTree']
    T_genieEvtRec = uproot.open(file)['GenieEvtRecTree']

    #—————————————————————————————————————
    # for each tree, create a data frame
    #—————————————————————————————————————
    
    
    #Reco dfs
    df_genieEvtRec = T_genieEvtRec.pandas.df(genieEvtRec_vars, flatten=False)
    df_rec = T_rec.pandas.df(rec_vars, flatten=False)
    
    #use concat to merge the individual reco dataframes
    df_ = pd.concat([df_genieEvtRec, df_rec], axis=1)

    # Add additional variables
    # ------------------------------------------------
    # CHANGE HERE TO ADD NEW VARIABLES TO DATAFRAME
    # ------------------------------------------------  
    df_ = calc_costheta_beam_dir(df_)
    df_ = calc_costheta_target(df_)
    df_ = GenieEvtRec_neutrino_pdg_var(df_)
    df_ = add_GenieEvtRec_iscc_var(df_)

    return df_
    
    

### Functions for adding extra vars into the dataframe

#### - Create Genie-Truth isCC var
#### - Creating Genie-Truth neutrino PDG var
#### - selection of primary muon and get their momentum separately from GenieTruth P4 var
#### - Lepton angle with neutrino direction estimated as the projection from the BNB target
#### - Lepton angle with neutrino direction estimated as the unit vector (0,0,1) in the beam direction (z)
#### -

In [ ]:
def add_GenieEvtRec_iscc_var(df):
    """
    Adds a scalar CC flag (0 or 1) from rec.mc.nu.iscc,
    which is stored as a 1-element array per event.
    Returns the full dataframe with the new column added.
    """

    df_ = df.copy()

    # Flatten rec.mc.nu.iscc (looks like [1], [0], ...)
    df_["GenieEvtRec_iscc"] = df_["rec.mc.nu.iscc"].str[0]

    return df_


In [ ]:
def GenieEvtRec_neutrino_pdg_var(df):
    """
    Create a new variable for the Genie-Truth neutrino PDG from the StdHepPdg array
    """
    df_ = df.copy()
    
    df_["GenieEvtRec_nuPdg"] = df_["GenieEvtRec.StdHepPdg"].str[0]
    
    return df_

In [ ]:
def get_primary_muon_first_from_row(row,
                                    pdg_col="GenieEvtRec.StdHepPdg",
                                    p4_col="GenieEvtRec.StdHepP4"):
    """
    Primary muon = first PDG with abs(pdg) == 13.
    Returns a Series (px, py, pz) or (nan, nan, nan) if no muon.
    """
    pdg_list = row[pdg_col]
    p4_list  = row[p4_col]

    # guard against missing data
    if pdg_list is None or p4_list is None:
        return pd.Series([np.nan, np.nan, np.nan],
                         index=["GenieEvtRec_mu_px", "GenieEvtRec_mu_py", "GenieEvtRec_mu_pz"])

    for i, pdg in enumerate(pdg_list):
        if abs(pdg) == 13:
            px, py, pz, E = p4_list[i]
            return pd.Series([px, py, pz],
                             index=["GenieEvtRec_mu_px", "GenieEvtRec_mu_py", "GenieEvtRec_mu_pz"])

    # no muon found in this event
    return pd.Series([np.nan, np.nan, np.nan],
                     index=["GenieEvtRec_mu_px", "GenieEvtRec_mu_py", "GenieEvtRec_mu_pz"])


In [ ]:
#===========================================================================#
# Angle between target→vertex direction and primary muon (GENIE truth muon) #
#===========================================================================#

def calc_costheta_target(df):
    #-----------------------------
    # 1) Primary muon px, py, pz
    #-----------------------------
    df[["GenieEvtRec_mu_px", "GenieEvtRec_mu_py", "GenieEvtRec_mu_pz"]] = df.apply(
        get_primary_muon_first_from_row, axis=1
    )

    #-----------------------------
    # 2) Vector from TARGET -> VERTEX
    #    target at (-74, 0, -110)
    #-----------------------------
    x_targ, y_targ, z_targ = -74.0, 0.0, -110.0

    # Get the vector components of the vtx position
    df.loc[:,'GenieEvtRec_nuvtxX'] = df["GenieEvtRec.EvtVtx[0]"]
    df.loc[:,'GenieEvtRec_nuvtxY'] = df["GenieEvtRec.EvtVtx[1]"]
    df.loc[:,'GenieEvtRec_nuvtxZ'] = df["GenieEvtRec.EvtVtx[2]"]


    df.eval(
        f"vec_targ_vtx_X = GenieEvtRec_nuvtxX - ({x_targ})",
        inplace=True
    )
    df.eval(
        f"vec_targ_vtx_Y = GenieEvtRec_nuvtxY - ({y_targ})",
        inplace=True
    )
    df.eval(
        f"vec_targ_vtx_Z = GenieEvtRec_nuvtxZ - ({z_targ})",
        inplace=True
    )

    #-----------------------------
    # 3) Norms of the two vectors
    #-----------------------------
    df.eval(
        "norm_vec_targ_vtx = sqrt(vec_targ_vtx_X**2 + vec_targ_vtx_Y**2 + vec_targ_vtx_Z**2)",
        inplace=True
    )
    df.eval(
        "norm_vec_mu = sqrt(GenieEvtRec_mu_px**2 + GenieEvtRec_mu_py**2 + GenieEvtRec_mu_pz**2)",
        inplace=True
    )

    #-----------------------------
    # 4) cos(theta) between them
    #   cosθ = (a·b) / (|a||b|)
    #-----------------------------
    df.eval(
        "cos_theta_bnb_target = (vec_targ_vtx_X * GenieEvtRec_mu_px"
        "           + vec_targ_vtx_Y * GenieEvtRec_mu_py"
        "           + vec_targ_vtx_Z * GenieEvtRec_mu_pz)"
        "          / (norm_vec_targ_vtx * norm_vec_mu)",
        inplace=True
    )

    # avoid 0/0
    mask_zero = (df["norm_vec_targ_vtx"] == 0) | (df["norm_vec_mu"] == 0)
    df.loc[mask_zero, "cos_theta_bnb_target"] = np.nan

    return df


In [ ]:
#===================================================================================#
# A Function for calculating the angle b/w beam direction axis and muon from vertex #
#===================================================================================#

def calc_costheta_beam_dir(df):
    
    # extract primary muon px, py, pz
    df[["GenieEvtRec_mu_px", "GenieEvtRec_mu_py", "GenieEvtRec_mu_pz"]] = df.apply(
        get_primary_muon_first_from_row, axis=1
    )
    
    # calculate the norm of the vector
    df.eval('GenieEvtRec_mu_pmag = sqrt(GenieEvtRec_mu_px**2 + GenieEvtRec_mu_py**2 + GenieEvtRec_mu_pz**2)', inplace=True)
    
    # beam is along +z → cosθ = pz / |p|
    df.eval(
        'cos_theta_beam_dir = GenieEvtRec_mu_pz / GenieEvtRec_mu_pmag',
        inplace=True
    )
    
    # avoid invalid 0/0
    df.loc[df["GenieEvtRec_mu_pmag"] == 0, "cos_theta_beam_dir"] = np.nan

    return df

### Functions for plotting

#### - MC category definition, including background
#### - Plotting function for stacked NON-area normalized distributions
#### - Functions to confirm the cuts and the vars applied

In [ ]:
# def isNue_NuebarCC(df):
#     df_ = df[((df.GenieEvtRec_nuPdg==12) | (df.GenieEvtRec_nuPdg==-12))    
#              & (df["rec.mc.nu.iscc"] == 1)]
#     return df_

# def isNumuCC(df):
#     df_ = df[(df.GenieEvtRec_nuPdg==14) &                             
#              (df["rec.mc.nu.iscc"] == 1)]
#     return df_

# def isNumu_barCC(df):
#     df_ = df[(df.GenieEvtRec_nuPdg==-14) &                             
#              (df["rec.mc.nu.iscc"] == 1)]
#     return df_

# def isNCpi(df):
#     # charged pion (±211) in final state
#     mask_pi = np.array([
#         211 in arr for arr in np.abs(np.array(df["GenieEvtRec.StdHepPdg"]))
#     ])
#     # NC = isCC == 0
#     mask_isNC = df["rec.mc.nu.iscc"] == 0
#     df_ = df[mask_isNC & mask_pi]
#     return df_

# def isNC(df):
#     # no charged pion
#     mask_no_pi = np.array([
#         211 not in arr for arr in np.abs(np.array(df["GenieEvtRec.StdHepPdg"]))
#     ])
#     # NC = isCC == 0
#     mask_isNC = df["rec.mc.nu.iscc"] == 0
#     df_ = df[mask_isNC & mask_no_pi]
#     return df_


In [ ]:
def isNumuCC(df):
    mask = (df["GenieEvtRec_nuPdg"] == 14) & (df["GenieEvtRec_iscc"] == 1)
    return df[mask]

def isNumu_barCC(df):
    mask = (df["GenieEvtRec_nuPdg"] == -14) & (df["GenieEvtRec_iscc"] == 1)
    return df[mask]

def isNue_NuebarCC(df):
    mask_flavour = (df["GenieEvtRec_nuPdg"] == 12) | (df["GenieEvtRec_nuPdg"] == -12)
    mask_cc      = (df["GenieEvtRec_iscc"] == 1)
    return df[mask_flavour & mask_cc]

def isNCpi(df):
    # NC with at least one charged pion (±211)
    mask_isNC = (df["GenieEvtRec_iscc"] == 0)
    mask_pi = df["GenieEvtRec.StdHepPdg"].apply(
        lambda lst: any(abs(p) == 211 for p in lst)
    ).to_numpy()
    return df[mask_isNC & mask_pi]

def isNC(df):
    # NC with NO charged pion (±211)
    mask_isNC = (df["GenieEvtRec_iscc"] == 0)
    mask_no_pi = df["GenieEvtRec.StdHepPdg"].apply(
        lambda lst: not any(abs(p) == 211 for p in lst)
    ).to_numpy()
    return df[mask_isNC & mask_no_pi]

In [ ]:
#=============================================#
#  Functions setting vars to plot after cuts  #
#=============================================#

def plots_for_SelectionCuts(df_, label, condition='default'):
    
    # this function makes all the plots that you want to plot throughout the pre-selection cuts
    
    if (condition=='default'):
        # --- MC/DATA comparison
        # x-dir reco neutrino vertex (also plot zoomed in edges)
        mc_data_stacked_hist(df_, "GenieEvtRec.EvtVtx[0]", \
                            "plots/%s_nuvtxX" % label, "True neutrino Vertex X [cm]", \
                            -200., 200, 25)
        
        # y-dir reco neutrino vertex (also plot zoomed in edges)
        mc_data_stacked_hist(df_, "GenieEvtRec.EvtVtx[1]", \
                            "plots/%s_nuvtxY" % label, "True neutrino Vertex Y [cm]", \
                            -200, 200, 25)
        # z-dir reco neutrino vertex (also plot zoomed in edges)
        mc_data_stacked_hist(df_, "GenieEvtRec.EvtVtx[2]", \
                            "plots/%s_nuvtxZ" % label, "True neutrino Vertex Z [cm]", \
                            0, 500, 25)


        # Flash Time (temporarily used as 1 bin histo)
        mc_data_stacked_hist(df_, "GenieEvtRec.EvtVtx[1]",\
                            "plots/%s_1_bin_histogram" % label, "1-Bin (A.U.)", \
                            -1000, 1000, 1)

        # # 1 bin histo with no category, data vs rest
        # mc_data_stacked_hist_no_cat(df_, "GenieEvtRec.EvtVtx[1]", \
        #                     "plots/%s_1" % label, "1-Bin (A.U.)", \
        #                     1000, 1000, 1)

    
        # calculated theta angle b/w beam and muon
        mc_data_stacked_hist(df_, "cos_theta_bnb_target",\
                            "plots/%s_cos_theta" % label, r"cos($\theta$) w.r.t the Projected BNB target",\
                            -1.2, 1.2, 25)
        
        mc_data_stacked_hist(df_, "cos_theta_beam_dir",\
                            "plots/%s_cos_theta" % label, r"cos($\theta$) w.r.t the Beam Direction",\
                            -1.2, 1.2, 25)
        

In [ ]:
#=============================================#
#  A Function for Plotting Stacked Histograms #
#=============================================#


def mc_data_stacked_hist(df_, var, plot_png_name, xaxis, xmin, \
                         xmax, nbins):
    #————————————————————————————————————————————————#
    # Restrict the range of df in the plotting range #
    #————————————————————————————————————————————————#
    
    
    # df_data = df_data[(df_data[var]>=xmin) & (df_data[var]<=xmax)]
    # df_ext = df_ext[(df_ext[var]>=xmin) & (df_ext[var]<=xmax)]
    # df_mc = df_mc[(df_mc[var]>=xmin) & (df_mc[var]<=xmax)]
    # df_dirt = df_dirt[(df_dirt[var]>=xmin) & (df_dirt[var]<=xmax)]
    
   
    
    #————————————————————————————————————————————#
    # Get the number of entries for each dataset #
    #————————————————————————————————————————————#
    
    n_mc = len(df_)

    #———————————————————————————————————#
    # Classify mc df into each topology #
    #———————————————————————————————————#
    
    # numuCC
    df_numuCC = isNumuCC(df_)
    n_numuCC = len(df_numuCC)

    # numubarCC
    df_numu_barCC = isNumu_barCC(df_)
    n_numu_barCC = len(df_numu_barCC)

    # NCpi+,-
    df_ncpi = isNCpi(df_)
    n_ncpi = len(df_ncpi)

    # NC
    df_nc = isNC(df_)
    n_nc = len(df_nc)
    
    # nue and nuebarCC
    df_nue_nuebarCC = isNue_NuebarCC(df_)
    n_nue_nuebarCC = len(df_nue_nuebarCC)

    print('\nOverlay (%i entries)' % n_mc)
    print('NumuCC %10i' % n_numuCC)
    print('NumubarCC    %10i' % n_numu_barCC)
    print('NCpi      %10i' % n_ncpi)
    print('NC        %10i' % n_nc)
    print('Nue/NuebarCC %10i' % n_nue_nuebarCC)
    print('Total     %10i' % (n_numuCC + n_numu_barCC \
                              + n_ncpi + n_nc + n_nue_nuebarCC))


    #—————————————#
    # get entries #
    #—————————————#
    
    hist_list = [df_numuCC[var],
                df_numu_barCC[var],
                df_nc[var],
                df_ncpi[var],
                df_nue_nuebarCC[var]]


    # #—————————————#
    # # get weight  #
    # #—————————————#
    
    # w_list = [df_cosmic[weight],
    #             df_outFV[weight],
    #             df_numuCC[weight],
    #             df_numu_barCC[weight],
    #             df_nc[weight],
    #             df_ncpi[weight],
    #             df_nue_nuebarCC[weight],
    #             df_ext[weight],
    #             df_dirt[weight]]


    #—————————————#
    # Colors!!!!! #
    #—————————————#
    
    c_list = ['paleturquoise',
            'red',
            'limegreen',
            'aqua',
            'orange']
    #         'saddlebrown',
    #         'grey',
    #         'gold',
    #         'pink']

    # c_data = 'black'
    
    
    #————————#
    # labels #
    #————————#
 
    label_list = [r'$\nu_{\mu}$ CC (%1.1f)'%(n_numuCC),
                r'$\bar{\nu}_{\mu}$ CC (%1.1f)'%(n_numu_barCC),
                'NC (%1.1f)'%(n_nc),
                r'NC $\pi$ (%1.1f)'%(n_ncpi),
                r'$\nu_{e}$ CC (%1.1f)'%(n_nue_nuebarCC)]
    
    #————————————————————————————#
    # Calculate Stat Uncertainty #
    #————————————————————————————#

    h_numuCC, b_numuCC = np.histogram(hist_list[0], bins=nbins, range=(xmin,xmax))
    h_numuBarCC, b_numuBarCC = np.histogram(hist_list[1], bins=nbins, range=(xmin,xmax))
    h_nc, b_nc = np.histogram(hist_list[2], bins=nbins, range=(xmin,xmax))
    h_ncPi, b_ncPi = np.histogram(hist_list[3], bins=nbins, range=(xmin,xmax))
    h_nueCC, b_nueCC = np.histogram(hist_list[4], bins=nbins, range=(xmin,xmax))

    # sum mc distributions
    total_mc = np.array([h_numuCC, h_numuBarCC, h_nc, h_ncPi, h_nueCC])
    total_mc = total_mc.sum(axis=0)
    total_mc_max = np.max(total_mc)

    # calculate their individual stat uncert as sqrt(number of entries) 
    stat_numuCC = np.sqrt(h_numuCC)
    stat_numuBarCC = np.sqrt(h_numuBarCC)
    stat_nc = np.sqrt(h_nc)
    stat_ncPi = np.sqrt(h_ncPi)
    stat_nueCC = np.sqrt(h_nueCC)

    total_stat = np.array([stat_numuCC, stat_numuBarCC, 
                        stat_nc, stat_ncPi,  stat_nueCC])
    total_stat = np.sqrt(total_stat.sum(axis=0))


    #——————————————————————#
    # Create Stacked Histo #
    #——————————————————————#
    # #xerr for uneven histo reco nu E
    # if (not isinstance(nbins, int)):
    #     xerr_reco_nuE = []
    #     for i in range(len(nbins)):
    #         print(len(nbins), i)
    #         if (i<len(nbins)-1):
    #             xerr_reco_nuE.append(0.5*(nbins[i+1]-nbins[i]))              
    
    fig, axs = plt.subplots(1, 1, figsize=(8,8))
    plt.subplots_adjust(left=0.125, bottom=0.1, right=0.9, top=0.9, wspace=0.2, hspace=0.1)
    axs.hist(hist_list, bins=nbins, range=(xmin,xmax), color=c_list, label=label_list, stacked=True)
    h_mc_max = np.max(total_mc_max)
    # err_bar = [np.sqrt(x) for x in h_data] # h_err = sqrt(bin)
    # mid = 0.5*(b_data[1:] + b_data[:-1])
    # if (not isinstance(nbins, int)):
    #     axs[0].errorbar(mid, h_data, xerr=xerr_reco_nuE, yerr=err_bar, color='black', label=label_data, fmt='o')
    # else:
    #     axs[0].errorbar(mid, h_data, xerr=0.5*(xmax-xmin)/nbins, yerr=err_bar, color='black', label=label_data, fmt='o')
    # upvals = np.append((np.array(total_mc)+np.array(total_stat)),(np.array(total_mc)+np.array(total_stat))[-1])
    # lowvals = np.append((np.array(total_mc)-np.array(total_stat)),(np.array(total_mc)-np.array(total_stat))[-1])
    # axs[0].fill_between(b_data, lowvals, upvals, step='post', color='gray', hatch='///', alpha=0.3, zorder=2)
    axs.legend(ncol=2, fontsize=12, frameon=False, loc='best')
    axs.set_ylabel('Entries', size=15)
    axs.set_xlabel('%s' % xaxis, size=15)
    axs.set_ylim([0, 1.5*h_mc_max])
  
    plt.savefig('%s.png' % plot_png_name, dpi=600)
    plt.tight_layout()
    plt.show()

## MAIN ANALYSIS CODE
### - Create the dataframe(s)
### - 

In [ ]:
#============================#
# Relevant files and paths   #
#============================#

file_RHC_BNB = "/Users/s2106059/Documents/sbnd/PSTF_truth/samples/prodgenie_v10_06_01_g4bnb_rhc_v2_1.flat.caf.root"

In [ ]:
df_RHC_BNB = new_df(file_RHC_BNB)

In [ ]:
df_RHC_BNB.head()

### Cutflow

#### - No cut, truth neutrino interactions
#### - Charge Current selection
#### - Vertex Containment, fiducial volume

In [ ]:
plots_for_SelectionCuts(df_RHC_BNB, 'RHC_BNB_all')
#df_RHC_BNB.to_hdf('dfs/df_RHC_BNB_no_selection.hdf5', key='df', mode='w')

In [ ]:
#==========================#
# Charge Current Selection #
#==========================#

df_RHC_BNB_CCSelection = df_RHC_BNB[(df_RHC_BNB['GenieEvtRec_iscc'] == 1)]
plots_for_SelectionCuts(df_RHC_BNB_CCSelection, 'RHC_BNB_CCSelection')
#df_RHC_BNB_CCSelection.to_hdf('dfs/df_RHC_BNB_CCSelection.hdf5', key='df', mode='w')

# BDT Stage

From the previous last selection, that's the sample that'll be used for BDT training. Due to truth study, there will be a binary BDT to separate between $\nu_\mu CC$ and $\bar{\nu}_\mu CC$. The only background will be the negligible mixed $\nu_e$ and $\bar{\nu}_e CC$ events

- Auxiliary functions used for BDT, mainly for labeling signals and creating BDT labels
- 

In [ ]:
#==================================#
# Auxiliary functions used for BDT #
#==================================#


def label_signal(df_):
    # this function labels the sample, =1 for signal (numuCC events), and =0 for background (otherwise) # flip definition
      
    condition_numubar = (df_['truth_nuPdg']==-14)
    condition_numu = (df_['truth_nuPdg']==14)
    condition_cc = (df_['truth_isCC']==1)

    condition_not_numubar = (df_['truth_nuPdg']!=-14)
    condition_not_numu = (df_['truth_nuPdg']!=14)
    condition_not_cc = (df_['truth_isCC']!=1)

    condition_not_allNumu_reco = (df_['numu_cc_flag']<1)
    
    query_numubarcc = condition_numubar & condition_cc
    query_numucc = condition_numu & condition_cc
    query_others = (condition_not_numubar | condition_not_numu) & condition_not_cc

    df_numucc_init = df_[(query_numucc)]
    df_numubarcc_init = df_[(query_numubarcc)]
    df_others_init = df_.drop(df_numucc_init.index, errors='ignore').drop(df_numubarcc_init.index, errors='ignore')
    totalEvents = len(df_numucc_init) + len(df_numubarcc_init) + len(df_others_init)
    
    df_["bdt_numubarcc_label"] = df_.apply(lambda row:1 if query_numubarcc[row.name] else 0, axis=1)
    df_["bdt_numucc_label"] = df_.apply(lambda row:1 if query_numucc[row.name] else 0, axis=1)
    df_["bdt_others_label"] = df_.apply(lambda row:1 if query_others[row.name] else 0, axis=1)

    #Calculate the percentage of 1's
    percentage_signal_numu = (len(df_numucc_init) / totalEvents) * 100
    print(f"Labelled dataset. Portion of signal numuCC events in this sample: {percentage_signal_numu:.2f}%")
    
    percentage_signal_numubar = (len(df_numubarcc_init) / totalEvents) * 100
    print(f"Labelled dataset. Portion of signal numubarCC events in this sample: {percentage_signal_numubar:.2f}%")

    percentage_others = (len(df_others_init) / totalEvents) * 100
    print(f"Labelled dataset. Portion of background events in this sample: {percentage_others:.2f}%")
    
def remove_background_events(df_):
    # remove any event that is not nueCC/nuebarCC
    query = "(truth_nuPdg==14 | truth_nuPdg==-14) & truth_isCC==1"
    num_events_before = len(df_)
    df_.query(query, inplace=True) # keep only events where query=True
    contaminant_contribution = (num_events_before - len(df_))/num_events_before
    print(f"Removed {num_events_before - len(df_)} contaminants from the selection sample. This"
          f" constitutes {contaminant_contribution:.2%} of the sample.")